## Description
This Jupyter Notebook generate the network overlapping the base map of Humanities and the publications of humanists to visualize the thematic differences between archetypes.

In [99]:
import pandas as pd
import random

# Data
- df_docs = with job titles
- dc_terms = with extracted keywords from the documents
- df_do_terms = with the relationship between documents and terms

In [100]:
df_docs = pd.read_csv('data/titles.tsv', sep='\t', quotechar='"', escapechar='\\')
df_terms = pd.read_csv('data/terms_all.tsv', sep='\t')
df_doc_terms = pd.read_csv('data/doc_terms_all.tsv', sep='\t')

In [105]:
df_doc_terms = df_doc_terms.merge(df_terms, left_on='term id', right_on='id')
df_doc_terms = df_doc_terms.merge(df_docs, left_on='document id', right_on='CODIGO OBRA DIALNET')
df_doc_terms = df_doc_terms[['CODIGO OBRA DIALNET', 'term id', 'term', 'TITULO']]
df_doc_terms

In [106]:
df_key = df_doc_terms[['CODIGO OBRA DIALNET', 'term']].drop_duplicates()
df_key

In [ ]:
df_aut = pd.read_csv('data/authors_publications_full_lang.tsv', sep='\t')
df_aut = df_aut[df_aut['DB']=='DIALNET']
df_aut = df_aut[['CODIGO AUTOR DIALNET', 'CODIGO OBRA DIALNET']].drop_duplicates()
df_aut

In [ ]:
df_aut_key_full = df_aut.merge(df_key, left_on='CODIGO OBRA DIALNET', right_on='CODIGO OBRA DIALNET')
df_aut_key_full

# Archetype overlay

In [109]:
df_arc = pd.read_csv('data/archetypes/philosophy.tsv', sep='\t')
df_arc

In [110]:
df_arc = df_arc.merge(df_aut, how='inner', left_on='dialnet_id', right_on='CODIGO AUTOR DIALNET')
df_arc = df_arc[['dialnet_id', 'archetype', 'CODIGO OBRA DIALNET']]
df_arc

In [111]:
df_aut_key = df_arc.merge(df_key, how='inner', on='CODIGO OBRA DIALNET')
df_aut_key = df_aut_key[['archetype', 'dialnet_id', 'term', 'CODIGO OBRA DIALNET']].drop_duplicates() #binary
df_aut_key_id = df_aut_key.copy()
df_aut_key.shape

In [112]:
nodes = pd.read_csv('data/networks/arc_vertex.txt', sep='\t')
nodes.rename({'name':'term'}, axis=1, inplace=True)
nodes

In [114]:
net_map = pd.read_csv('data/networks/arc_map.txt', sep='\t')
net_map

In [115]:
nodes_total = nodes.merge(df_aut_key_full, how='left', on='term')
nodes_total

In [118]:
nodes_total = nodes_total[['cluster', 'CODIGO OBRA DIALNET']].drop_duplicates().dropna()
nodes_total = nodes_total.groupby('cluster').size().reset_index().rename({0:'Outputs'}, axis=1)
nodes_total

In [119]:
for n in df_aut_key.archetype.drop_duplicates():
    nodes_c = nodes.merge(df_aut_key[df_aut_key.archetype==n], how='left', on='term')
    nodes_c = nodes_c[['cluster', 'CODIGO OBRA DIALNET']].drop_duplicates().dropna()
    total_arc = len(nodes_c['CODIGO OBRA DIALNET'].drop_duplicates())
    nodes_c = nodes_c.groupby('cluster').size().reset_index().rename({0:'Freq'}, axis=1)

    nodes = nodes.merge(nodes_c, how='left', on='cluster')
    nodes['Freq'].fillna(0, inplace=True)
    nodes['Freq'] = 100*nodes['Freq']/total_arc
    nodes.rename({'Freq':'weight<arch'+str(n)+'>'}, axis=1, inplace=True)
nodes

In [120]:
nodes = nodes.drop(['term', 'degree'], axis=1).drop_duplicates()
nodes = nodes.merge(nodes_total, how='left', on='cluster')
nodes.rename({'cluster':'id'}, axis=1, inplace=True)
nodes

In [121]:
net_map_arc = net_map.merge(nodes, how='left', on='id')
net_map_arc.rename({'Outputs':'weight<Outputs>'}, axis=1, inplace=True)
net_map_arc.to_csv('data/networks/arc_map_philosophy.txt', sep='\t', index=False)